In [ ]:
!pip install keras==2.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 2.12.0 which is incompatible.


In [1]:
import pandas as pd
import io
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn  # neural net layers and activations
from torch.optim import SGD  # Our chosen optimizer
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler

from keras.wrappers.scikit_learn import KerasRegressor

from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam

In [2]:
X = pd.read_csv('LM_Data.csv', delimiter =',') #training data
y = pd.read_csv('selectivity_Data.csv',delimiter=',') #true output labels, original

In [3]:
print(X.shape, y.shape)
X.head()

(20460, 40) (20460, 1)


,Elevation <,Elevation =,Elevation >,Elevation values,Aspect <,Aspect =,Aspect >,Aspect values,Slope <,Slope =,...,Hillshade_Noon >,Hillshade_Noon values,Hillshade_3pm <,Hillshade_3pm =,Hillshade_3pm >,Hillshade_3pm values,Horizontal_Distance_To_Fire_Points <,Horizontal_Distance_To_Fire_Points =,Horizontal_Distance_To_Fire_Points >,Horizontal_Distance_To_Fire_Points values
0,0,0,1,2954,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,3031,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,3380,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,1,2905,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,0,2919,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
scaler = MinMaxScaler()
cols = ['Elevation values','Aspect values', 'Slope values', 'Horizontal_Distance_To_Hydrology values','Vertical_Distance_To_Hydrology values', 'Horizontal_Distance_To_Roadways values', 'Hillshade_9am values','Hillshade_Noon values', 'Hillshade_3pm values', 'Horizontal_Distance_To_Fire_Points values']
X[cols] = scaler.fit_transform(X[cols])

#convert to numpy arrays
X = X.values
y = y.values #uaing this for model training

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 11) #testing 20%, trainig 80

In [6]:
def create_model(width=1, depth=1):
    model = Sequential()
    neurons = width
    model.add(Dense(neurons, input_shape=(40,), activation='relu'))
    for _ in range(depth-1):
        neurons = max(neurons // 2, 16)
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    print(model)
    return model

In [7]:
keras_regressor = KerasRegressor(build_fn = create_model, epochs=50)

# Define the grid search parameters
width = [32, 64, 128, 256]
depth = [1, 2, 3]
param_grid = dict(width=width, depth=depth)

# Perform grid search
grid = GridSearchCV(estimator=keras_regressor, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X_train, y_train)  # Assuming you have X_train and y_train

<ipython-input-7-ca2285674581>:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_regressor = KerasRegressor(build_fn = create_model, epochs=50)
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/50
512/512 [==============================] - 4s 6ms/step - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 2/50
512/512 [==============================] - 4s 7ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 3/50
512/512 [==============================] - 3s 6ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 4/50
512/512 [==============================] - 3s 5ms/step - loss: 0.0027 - mean_squared_error: 0.0027
Epoch 5/50
512/512 [==============================] - 3s 5ms/step - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 6/50
512/512 [==============================] - 3s 6ms/step - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 7/50
512/512 [==============================] - 3s 7ms/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 8/50
512/512 [==============================] - 3s 5ms/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 9/50
512/512 [==============================] - 3s 5ms/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 10/5

In [ ]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
from pprint import pprint

pprint(clf.grid_scores_)